In [1]:
#importing necessary libraries
import sys
import random
import copy
import numpy as np
from numpy.linalg import norm

class City:
    def __init__(self, index: int, distance: int):
        self.index = index
        self.distance = distance

    def __lt__(self, other):
        return self.distance < other.distance


class State:
    def __init__(self, route: list, distance: int = 0):
        self.route = route
        self.distance = distance

    def __eq__(self, other):
        for i in range(len(self.route)):
            if self.route[i] != other.route[i]:
                return False
        return True

    def __lt__(self, other):
        return self.distance < other.distance

    def __repr__(self):
        return "({0},{1})\n".format(self.route, self.distance)

    def copy(self):
        return State(self.route, self.distance)

    def deepcopy(self):
        return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))

    def update_distance(self, matrix, home):
        self.distance = 0
        from_index = home
        for i in range(len(self.route)):
            self.distance += matrix[from_index][self.route[i]]
            from_index = self.route[i]
        self.distance += matrix[from_index][home]


def probability(p):
	return p > random.uniform(0.0, 1.0)

def exp_schedule(k=20, lam=0.005, limit = 1000):
	return lambda t: (k * np.exp(-lam * t) if t < limit else 0)

def get_random_solution(matrix: list, home: int, city_indexes: list, size: int):
    cities = city_indexes.copy()
    cities.pop(home)
    population = []
    for i in range(size):
        random.shuffle(cities)
        state = State(cities[:])
        state.update_distance(matrix, home)
        population.append(state)
    return min(population)

def mutate(matrix: list, home: int, state: State, mutation_rate: float = 0.01):
    mutated_state = state.deepcopy()
    for i in range(len(mutated_state.route)):
        if random.random() < mutation_rate:
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    mutated_state.update_distance(matrix, home)
    return mutated_state

def simulated_annealing(matrix:list, home:int, initial_state:State, mutation_rate:float=0.01, schedule=exp_schedule()):
	best_state = initial_state
	for t in range(sys.maxsize):
		T = schedule(t)
		if T == 0:
			return best_state
		neighbor = mutate(matrix, home, best_state, mutation_rate)
		delta_e = best_state.distance - neighbor.distance
		if delta_e > 0 or probability(np.exp(delta_e / T)):
			return neighbor

def get_euclidean_distance(p, q):
    return round(norm(np.array(p) - np.array(q)))


if __name__ == "__main__":
    all_cities_coordinates = {1: (2,2), 2: (7, 3), 3: (9, 39), 4: (1,10)} #4 cities having the respective coordinates
    D = []
    for _, goal_coordinates in all_cities_coordinates.items():
        distances = []
        for _, coordinates in all_cities_coordinates.copy().items():
            distances.append(get_euclidean_distance(goal_coordinates, coordinates))
        D.append(distances)

    current = 0
    cities = list(all_cities_coordinates.keys())
    city_indexes = [index - 1 for index in cities]

    state = get_random_solution(D, current, city_indexes, 100)
    print("Travelling Salesperson Problem using simulated annealing\n")
    print()
    destination_state = simulated_annealing(D, current, state, 0.1)
    print("-- Simulated annealing solution --")
    print(cities[current], end="")
    for i in range(0, len(destination_state.route)):
        print(" -> " + str(cities[destination_state.route[i]]), end="")
    print(" -> " + str(cities[current]), end="")
    print("\n\nTotal distance: {0} miles".format(destination_state.distance))
    print()







Travelling Salesperson Problem using simulated annealing


-- Simulated annealing solution --
1 -> 2 -> 3 -> 4 -> 1

Total distance: 79 miles



Travelling Salesperson Problem using simulated annealing


-- Simulated annealing solution --
1 -> 4 -> 3 -> 2 -> 1

Total distance: 79 miles

